In [ ]:
# !pip install -qU selenium webdriver-manager persiantools python-docx

In [ ]:
# !pip install pandas

In [3]:
import csv
import re
import json
import logging
from tqdm import tqdm

import numpy as np
import pandas as pd

import docx

from selenium import webdriver

In [4]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36")

driver = webdriver.Chrome("chromedriver", options=options)

In [ ]:
class JsonFileIterator:
    def __init__(self, path):
        self.path = path
        self.f = open(path, "r")
        self.i = 0
        self.length = self.counter_lines()

    def __iter__(self):
        return self

    def __next__(self):
        line = self.f.readline()
        if not line:
            # End of file
            self.f.close()
            raise StopIteration
        self.i += 1
        return json.loads(line)

    def counter_lines(self):
        with open(self.path, "r") as f1:
            return sum(1 for _ in f1)

    def __len__(self):
        return self.length

In [ ]:
def read_json_lines(path, n_lines=None):
    """Creates a generator which reads and returns lines of
    a json lines file, one line at a time, each as a dictionary.

    This could be used as a memory-efficient alternative of `pandas.read_json`
    for reading a json lines file.
    """
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if n_lines == i:
                break
            yield json.loads(line)

In [ ]:
search_query = pd.DataFrame(read_json_lines('./data/torob-search-data_v1.jsonl'))
search_query.head()

In [ ]:
queries = dict()
for search in tqdm(search_query.to_dict('records')):
    query = search["raw_query"]
    if query not in queries:
        queries[query] = 1
    else:
        queries[query] += 1

In [ ]:
sorted_queries = sorted(queries.items(), key=lambda x: x[1], reverse=False)

In [ ]:
df = pd.DataFrame.from_dict(queries, orient='index', columns=['count'])
df.sort_values(by=['count'], inplace=True, ascending=True)

In [ ]:
df

In [ ]:
temp = df.copy(deep=True)
temp = temp[temp['count'] <= 10]
# temp[temp.index == "دچرخ"]

In [ ]:
temp.sort_values(by=['count'], inplace=True, ascending=False)

In [ ]:
temp

In [ ]:
dfs = np.array_split(temp, len(temp)//50000 + 1)

In [ ]:
# dfs = []

# # loop over each output file and read it into a DataFrame
# for i in range(1, n_files + 1):
#     filename = f"output_{i}.csv"
#     df = pd.read_csv(filename, index_col=0)
#     dfs.append(df)

# # concatenate the DataFrames into a single DataFrame
# temp = pd.concat(dfs)

In [ ]:
# !mkdir spell_check

In [ ]:
for i, df in enumerate(dfs):
    df.to_csv(f"./spell_check/output_{i+1}.csv")

In [ ]:
# create an empty list to store the DataFrames
dfs = []

# loop over each output file and read it into a DataFrame
for i in range(1, n_files + 1):
    filename = f"output_{i}.csv"
    df = pd.read_csv(filename, index_col=0)
    dfs.append(df)

# concatenate the DataFrames into a single DataFrame
temp = pd.concat(dfs)


In [ ]:
doc = docx.Document()

In [ ]:
temp = temp[:100]

In [ ]:
# آب میوه گیری چهار

In [ ]:
temp

In [ ]:
table = doc.add_table(rows=1, cols=len(temp.columns))
header_cells = table.rows[0].cells
for i in range(len(temp.columns)):
    header_cells[i].text = temp.columns[i]

for i, (index, row) in tqdm(enumerate(temp.iterrows()), total=len(temp)):
    cells = table.add_row().cells
    cells[0].text = str(index)  # write the index value to the first cell
    for j in range(1, len(temp.columns)):
        cells[j].text = str(row[j]).replace('\n', ' ')
        
# save file
doc.save("output.docx")

In [ ]:
df = pd.read_csv('./spell_check/output_2.csv', index_col=0)

In [ ]:
dfs = np.array_split(df, len(df)//10000 + 1)

In [ ]:
for i, df in enumerate(dfs):
    df.to_csv(f"./spell_check/output_2_{i+1}.csv")

In [ ]:
df = pd.read_csv('./spell_check/output_2_1.csv', index_col=0)

In [ ]:
temp = df.copy(deep=True)

In [ ]:
temp

In [ ]:
with open("retrive_part_2_1.txt", "w", encoding="utf-8", newline="") as csvfile:

    wrtiter = csv.writer(csvfile)
    wrtiter.writerow(
        [
            "query",
            "spell_checked",
            "title_text",
        ]
    )

    for query in tqdm(temp.index):

        driver.get("https://torob.com/search/?query=" + query)

        check_spell = driver.find_elements("xpath", "//p[contains(@class, 'black_p')]")

        torob_spell_check = False
        corrected_form = None
        for text in check_spell:
            extracted_text = text.get_attribute("innerText")
            if extracted_text.startswith("جستجو غلط‌گیری شد:"):
                torob_spell_check = True
                corrected_query = re.search(
                    r"جستجو غلط‌گیری شد: (.*)", extracted_text
                ).group(1)
                corrected_form = corrected_query
            else:
                corrected_form = None

        names = driver.find_elements(
            "xpath", "//h2[contains(@class,'jsx-fa8eb4b3b47a1d18')]"
        )
        name_list = []
        for name in names:
            extracted_name = name.get_attribute("innerText")
            name_list.append(extracted_name)

        title_text = "\n".join(name_list)

        wrtiter.writerow(
            [
                query,
                corrected_form,
                title_text,
            ]
        )


In [ ]:
# retrive_part_2_1.txt
# temp = temp[8012:]

In [ ]:
# temp.to_csv(f"./spell_check/output_2_1_part2.csv")

In [5]:
temp = pd.read_csv(f"./spell_check/output_2_1_part2.csv", index_col=0)

In [6]:
temp

,count
H500,3
مته کبالت,3
aux میکروفون دار,3
iPad 8,3
باکس پاستیل,3
...,...
دسته بازی بیسیم کامپیوتر,3
mi 11 lite ne,3
ساعت مچی زنانه طلایی,3
گوشی10,3


In [7]:
with open("retrive_part_2_1_part2.txt", "w", encoding="utf-8", newline="") as csvfile:

    wrtiter = csv.writer(csvfile)
    wrtiter.writerow(
        [
            "query",
            "spell_checked",
            "title_text",
        ]
    )

    for query in tqdm(temp.index):

        driver.get("https://torob.com/search/?query=" + query)

        check_spell = driver.find_elements("xpath", "//p[contains(@class, 'black_p')]")

        torob_spell_check = False
        corrected_form = None
        for text in check_spell:
            extracted_text = text.get_attribute("innerText")
            if extracted_text.startswith("جستجو غلط‌گیری شد:"):
                torob_spell_check = True
                corrected_query = re.search(
                    r"جستجو غلط‌گیری شد: (.*)", extracted_text
                ).group(1)
                corrected_form = corrected_query
            else:
                corrected_form = None

        names = driver.find_elements(
            "xpath", "//h2[contains(@class,'jsx-fa8eb4b3b47a1d18')]"
        )
        name_list = []
        for name in names:
            extracted_name = name.get_attribute("innerText")
            name_list.append(extracted_name)

        title_text = "\n".join(name_list)

        wrtiter.writerow(
            [
                query,
                corrected_form,
                title_text,
            ]
        )


100%|███████████████████████████████████████| 1660/1660 [52:56<00:00,  1.91s/it]
